In [14]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import string
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

#1.Downloadong the text


In [3]:
f = open('/content/11-0.txt', encoding='utf8').read()


#2. Preproccessing

In [4]:
# Converting to lower case and deleting end of strings
f = f.replace('\n', '').replace("’", '').replace("“", '').lower()


In [6]:
# Split text into tokens
tokens = pd.Series(word_tokenize(f))
# deleting stopwords
stop_words = stopwords.words("english")
tokens = tokens.loc[~tokens.isin(stop_words)]
tokens = tokens.loc[~tokens.isin(list(string.punctuation + string.digits + "’"  + "'" + '"' + '”' + '“' + '’' + '‘'))]

In [7]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
# Lemmatization
lemmatizer = WordNetLemmatizer()
tokens = tokens.apply(lambda w: lemmatizer.lemmatize(w))


In [8]:
#grouping tokens in whole text into chapters and converting it to Dataframe
chapters = []

chapter_nmb = 0
for word in tokens.iteritems():
  if (word[1] == 'chapter'):
    if chapter_nmb == 12:
      chapter_nmb = 1
    else:
      chapter_nmb += 1
  chapters.append({ 'chapter': chapter_nmb, 'word': word[1] })

chapter_text = pd.DataFrame(chapters)

#3. Here we'r finding top-10 words in every chapter

In [9]:
def topWords(df, chapter):
  current_chapter = df[df.chapter == chapter]
  unique_words = current_chapter[current_chapter.word != 'alice'].word.unique()

  words_count = []

  for w in unique_words:
    words_count.append({
      'word': w,
      'count': current_chapter.loc[current_chapter.word == w].word.count()
    })
  
  counted = pd.DataFrame(words_count)
  return counted.sort_values(by=['count'], ascending=False).head(10)

for i in range(12):
  i +=1
  print('Top 10 words in chapter ' + str(i))
  print(topWords(chapter_text, i))
  print('----------------------')

Top 10 words in chapter 1
          word  count
44        said    150
315      queen     46
121       went     30
13      hatter     29
15    dormouse     27
1132      mock     25
59        dont     23
25        head     23
32         one     21
84        time     21
----------------------
Top 10 words in chapter 2
          word  count
27        said     40
7         mock     26
8       turtle     24
28     gryphon     21
512       soup     11
190      would     11
520  beautiful     10
179       wont     10
5      lobster     10
66       dance      9
----------------------
Top 10 words in chapter 3
         word  count
80       said     36
8        king     20
218    hatter     16
240  dormouse     10
154       one      9
68      court      9
9       queen      9
33     rabbit      8
55    thought      8
215   witness      8
----------------------
Top 10 words in chapter 4
             word  count
674       project     65
112          work     64
65           said     48
762  gutenbe

#4. Here we are finding top-10 verbs in sent with Alce. We should split text not into chapters, but by symbols '.'

In [11]:
f = f.replace('”', '')
sentences =  f.split('.')
sentences_tokenized = [word_tokenize(sent) for sent in sentences]
len(sentences_tokenized)

1223

In [22]:
alice_list = []
k = 0
for sent in sentences_tokenized:
  k = 0
  for word in sent:
    if word =='alice':
      k +=1
  if k>0:
    alice_list.append(sent)

len(alice_list)

342

In [36]:
filter = []
for sent in alice_list:
  sent = [token for token in sent if token not in stop_words]
  sent = [lemmatizer.lemmatize(token, pos = 'v') for token in sent]
  postag = nltk.pos_tag(sent)
  for w in postag:
    if (w[1] == 'VB') or (w[1] == 'VBP'):
      filter.append(w[0])

In [40]:
print(filter)

['think', 'say', 'think', 'occur', 'wonder', 'seem', 'awatch', 'burn', 'run', 'see', 'alarge', 'go', 'consider', 'go', 'like', 'find', 'think', 'shallthink', 'brave', 'think', 'say', 'let', 'wouldbe', 'see', 'say', 'wonder', 'think', 'say', 'go', 'say', 'turn', 'hangingfrom', 'come', 'try', 'find', 'kneel', 'wander', 'cool', 'get', 'go', 'little', 'see', 'begin', 'think', 'find', 'find', 'say', 'say', 'wise', 'go', 'find', 'sort', 'say', 'shut', 'see', 'end', 'know', 'say', 'go', 'find', 'decide', 'get', 'find', 'find', 'reachit', 'see', 'try', 'try', 'use', 'say', 'advise', 'leave', 'generallygave', 'follow', 'bring', 'remember', 'try', 'pretend', 'think', 'pretend', 'leave', 'find', 'seem', 'get', 'manage', 'bekind', 'think', 'walk', 'go', 'see', 'ill', 'give', 'lie', 'tolook', 'get', 'say', 'say', 'go', 'go', 'tear', 'aboutfour', 'reach', 'begin', 'go', 'take', 'go', 'go', 'beenchanged', 'say', 'say', 'come', 'improve', 'seem', 'say', 'mabel', 'andi', 'go', 'learn', 'say', 'look', '

In [49]:
top_verbs = []
for w in filter:
  counter = 1
  for v in filter:
    if w == v:
      counter +=1
      filter.remove(v)
  top_verbs.append((w, counter))

In [50]:
sorted(top_verbs, key=lambda v: -v[1])[:10]

[('say', 176),
 ('go', 71),
 ('think', 60),
 ('get', 38),
 ('see', 31),
 ('say', 31),
 ('know', 27),
 ('make', 27),
 ('find', 23),
 ('take', 21)]